In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
nltk.download('stopwords')
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

[nltk_data] Error loading stopwords: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


In [2]:
dataset = pd.read_csv("datasets/toxic_comments_1.csv",nrows=6000)

In [3]:
dataset = dataset[["comment_text","toxic"]]

In [4]:
#cleaning the text
dataset.columns = ["comment","toxic"]
dataset["comment"] = dataset["comment"].str.replace("\n"," ")

In [5]:
bag_of_words = []
import re
for i in range(0,6000):
    words = re.sub('[^a-zA-Z]',' ',dataset["comment"][i])
    words = words.lower().split()
    words = [word for word in words if not word in set(stopwords.words("english"))]
    sentence = ' '.join(words)
    bag_of_words.append(sentence)

In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Embedding

In [29]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=20000)
tokenizer.fit_on_texts(np.array(bag_of_words))


In [30]:
from tensorflow.keras.preprocessing import sequence
X = tokenizer.texts_to_sequences(np.array(bag_of_words))
X = sequence.pad_sequences(X,maxlen=50)

In [31]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=0)

In [38]:
from tensorflow.keras.layers import Embedding
model = Sequential()
model.add(Embedding(20000,50))
model.add(Dense(50, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary() 

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, None, 50)          1000000   
_________________________________________________________________
dense_11 (Dense)             (None, None, 50)          2550      
_________________________________________________________________
dense_12 (Dense)             (None, None, 1)           51        
Total params: 1,002,601
Trainable params: 1,002,601
Non-trainable params: 0
_________________________________________________________________


In [39]:
#functions to calculate recall,precision and f1_score
from tensorflow.keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy',f1_m,precision_m, recall_m])

In [40]:
history = model.fit(X_train,y_train,batch_size=50,epochs=8,verbose=2)

Train on 4800 samples
Epoch 1/8
4800/4800 - 6s - loss: 0.4833 - accuracy: 0.8909 - f1_m: 0.1907 - precision_m: 0.3192 - recall_m: 1.0952
Epoch 2/8
4800/4800 - 4s - loss: 0.3154 - accuracy: 0.9058 - f1_m: 0.5214 - precision_m: 0.6637 - recall_m: 0.5452
Epoch 3/8
4800/4800 - 4s - loss: 0.2943 - accuracy: 0.9075 - f1_m: 1.0214 - precision_m: 0.7750 - recall_m: 1.8475
Epoch 4/8
4800/4800 - 4s - loss: 0.2856 - accuracy: 0.9109 - f1_m: 1.2254 - precision_m: 0.7367 - recall_m: 4.1049
Epoch 5/8
4800/4800 - 5s - loss: 0.2816 - accuracy: 0.9109 - f1_m: 1.2115 - precision_m: 0.7201 - recall_m: 4.4564
Epoch 6/8
4800/4800 - 5s - loss: 0.2787 - accuracy: 0.9109 - f1_m: 1.1686 - precision_m: 0.6882 - recall_m: 4.7535
Epoch 7/8
4800/4800 - 5s - loss: 0.2772 - accuracy: 0.9110 - f1_m: 1.2068 - precision_m: 0.7088 - recall_m: 4.7939
Epoch 8/8
4800/4800 - 5s - loss: 0.2765 - accuracy: 0.9109 - f1_m: 1.2052 - precision_m: 0.7069 - recall_m: 4.8220


In [41]:
y_pred = model.predict(X_test)

In [45]:
loss, accuracy, f1_score, precision, recall = model.evaluate(X_test,y_test,batch_size=50,verbose=2)

1200/1200 - 1s - loss: 0.3736 - accuracy: 0.8788 - f1_m: 0.8250 - precision_m: 0.5039 - recall_m: 2.5465


In [53]:
y_pred = model.predict(X_test)

In [54]:
len(y_pred)

1200

In [58]:
results = []
for result in y_pred:
    answer = max(result)
    if answer > 0.5:
        results.append(1)
    else:
        results.append(0)
results = np.array(results)

In [59]:
len(results)

1200